# Tworzy model zawierający nazwy ulic w całej Polsce

Utworzony model wyszukuje tylko podane wzorce - nie bazuje jeszcze na sztucznej inteligencji.

In [43]:
from spacy.lang.pl import Polish
import json

In [44]:
import os
src_dir = os.path.join("..", "data")
out_dir = os.path.join("..", "model")
src_streets = os.path.join(src_dir, "streets.json")
out_streets = os.path.join(out_dir, "pl_streets_ner")

In [45]:
def load_json_data(file: str):
    with open(file, 'tr', encoding="UTF-8") as json_file:
        data = json.load(json_file)
    return data

def save_json_data(file: str, json_data):
    with open(file, 'tw', encoding="UTF-8") as outfile:
        json.dump(json_data, outfile, indent=4, ensure_ascii=False)

In [46]:
def generate_streets(file: str) -> set:
    streets = load_json_data(file)
    ret = set()
    for s in streets:
        name = s.get("name")
        if (len(name) > 3 ):
            ret.add(name)
        if s.get("prefix") != None:
            ret.add(s.get("prefix") + " " + s.get("name"))
        if s.get("name_ext") != None:
            ret.add(s.get("name_ext") + " " + s.get("name"))
        if s.get("prefix") != None and s.get("name_ext") != None:
            ret.add(s.get("prefix") + " " + s.get("name_ext") + " " + s.get("name"))
    return ret

Tworzy listę wzorców w odpowiednim formacie.

In [47]:
def generate_patterns(data: list, type: str) -> list:
    patterns = []
    for item in data:
        pattern = {"label": type, "pattern": item}
        patterns.append(pattern)
    return patterns

Tworzy model na podstawie listy wzorców i zapisuje na dysku.

In [48]:
def generate_rules(patterns, model_dir: str):
    nlp = Polish()
    ruler = nlp.add_pipe("entity_ruler")
    ruler.add_patterns(patterns)
    nlp.to_disk(model_dir)


streets = generate_streets(src_streets)
# save_json_data("str.json", list(streets))
patterns = generate_patterns(streets, "STREET")
generate_rules(patterns, out_streets)

## Testowanie wygenerowanego modelu

In [49]:
import spacy

nlp = spacy.load(out_streets)

In [52]:
articles = load_json_data("../data/trojmiasto.json")
ents = set()
for a in articles:
    body = a.get("body")
    doc = nlp(body)
    for ent in doc.ents:
        ents.add(ent.text)
print(len(ents))
print(ents)

573
{'Edwarda', 'Jezioro', 'Park', 'Fortuna', 'Nowości', 'Hali', 'Ireny', 'Jana Pawła II', 'Michała', 'Bielska', 'Dworzec', 'Potok', 'Tomasza', 'ul. Korzeniowskiego', 'Kaczyńskiego', 'Wielkopolska', 'Rola', 'ul. Króla Jana Kazimierza', 'św. Dominika', 'Pomorskich', 'Wołek', 'Ścieżka', 'Przybysz', 'Powstania Styczniowego', 'Gdynia', 'Sławomira', 'Kacpra', 'J. Słowackiego', 'Rutki', 'Samodzielna', 'Przebieg', 'Marcin', 'św. Jana', 'ul. Wendy', 'Staniszewskiego', 'Medyczna', 'Litwa', 'Nowa', 'Socjalna', 'Bożeny', 'ul. Jałowcowa', 'Grodzisko', 'Główna', 'Pluty', 'Prus', 'Faustyny', 'Armii Krajowej', 'Nasza', 'Eryka', 'Bergera', 'Andrzeja', 'Łączna', 'Inna', 'Tadeusza Wendy', 'Teatralny', 'Letnie', 'Żurawie', 'Seweryna', 'ul. Na Zaspę', 'Lecha Bądkowskiego', 'św. Mikołaja', 'Agaty', 'ul. Jana z Kolna', 'Unii Europejskiej', 'Przerwa', 'Morska', 'sierpnia', 'Moja', 'Iwan', 'Stanisława Fijałkowskiego', 'Młoda', 'Żwirki', 'Najświętszej Maryi Panny', 'Wigury', 'Łokietka', 'Konstelacji', 'Sportow